# IPL Machine Learning Analysis

This notebook focuses on predictive analysis using scikit-learn.

In [9]:
# Import required libraries
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Window
import pyspark.sql.functions as F

# For ML
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("IPL ML Analysis") \
    .master("local[*]") \
    .getOrCreate()

# Load the data
deliveries_df = spark.read.csv('data/deliveries.csv', header=True, inferSchema=True)
matches_df = spark.read.csv('data/matches.csv', header=True, inferSchema=True)

## 1. Feature Engineering

In [10]:
# Create features for score prediction
def create_features_first_innings(df_deliveries, df_matches):
    # Join dataframes
    combined_df = df_deliveries.join(df_matches, df_deliveries.match_id == df_matches.id)
    
    # Calculate features for first 6 overs
    powerplay_stats = combined_df.filter(F.col('over') <= 6) \
        .groupBy('match_id') \
        .agg(F.sum('total_runs').alias('powerplay_runs'),
             F.sum('is_wicket').alias('powerplay_wickets'),
             F.count(F.when(F.col('extras_type').isNotNull(), 1)).alias('powerplay_extras'))
    
    # Calculate final innings score
    final_scores = combined_df.groupBy('match_id') \
        .agg(F.sum('total_runs').alias('final_score'))
    
    # Combine features
    feature_df = powerplay_stats.join(final_scores, 'match_id')
    
    return feature_df

# Create features for match winner prediction
def create_features_match_winner(df_matches):
    # Calculate team form (last 3 matches)
    matches_pd = df_matches.toPandas()
    matches_pd['team1_last3'] = matches_pd.groupby('team1')['winner'].transform(
        lambda x: x.shift().rolling(3).mean())
    matches_pd['team2_last3'] = matches_pd.groupby('team2')['winner'].transform(
        lambda x: x.shift().rolling(3).mean())
    
    return matches_pd

# Create feature datasets
score_features = create_features_first_innings(deliveries_df, matches_df)
# Fix: Ensure 'winner' is numeric before calculating rolling mean
matches_pd = matches_df.toPandas()
matches_pd['winner_numeric'] = (matches_pd['winner'] == matches_pd['team1']).astype(int)

matches_pd['team1_last3'] = matches_pd.groupby('team1')['winner_numeric'].transform(
    lambda x: x.shift().rolling(3).mean())
matches_pd['team2_last3'] = matches_pd.groupby('team2')['winner_numeric'].transform(
    lambda x: x.shift().rolling(3).mean())

winner_features = matches_pd

print("\n--- Score Prediction Features Sample ---")
score_features.show(5)

print("\n--- Match Winner Features Sample ---")
print(winner_features.head())


--- Score Prediction Features Sample ---
+--------+--------------+-----------------+----------------+-----------+
|match_id|powerplay_runs|powerplay_wickets|powerplay_extras|final_score|
+--------+--------------+-----------------+----------------+-----------+
|  419157|            76|                3|               5|        325|
|  392222|           105|                5|               8|        241|
|  419142|           102|                2|               7|        306|
|  392218|            94|                4|               9|        298|
|  336036|            89|                2|               7|        291|
+--------+--------------+-----------------+----------------+-----------+
only showing top 5 rows


--- Match Winner Features Sample ---
       id   season        city        date match_type player_of_match  \
0  335982  2007/08   Bangalore  2008-04-18     League     BB McCullum   
1  335983  2007/08  Chandigarh  2008-04-19     League      MEK Hussey   
2  335984  2007/08 

## 2. Final Score Prediction Model

In [11]:
# Prepare data for score prediction
score_data = score_features.toPandas()

X = score_data[['powerplay_runs', 'powerplay_wickets', 'powerplay_extras']]
y = score_data['final_score']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train model
score_model = RandomForestRegressor(n_estimators=100, random_state=42)
score_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = score_model.predict(X_test_scaled)

# Evaluate model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f'Root Mean Square Error: {rmse}')

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': score_model.feature_importances_
})
print("\nFeature Importance:")
print(feature_importance.sort_values('importance', ascending=False))

Root Mean Square Error: 51.052375270792226

Feature Importance:
             feature  importance
0     powerplay_runs    0.642684
2   powerplay_extras    0.193509
1  powerplay_wickets    0.163807


## 3. Match Winner Prediction Model

In [12]:
# Prepare data for winner prediction
winner_data = winner_features.dropna()

X = winner_data[['team1_last3', 'team2_last3', 'toss_winner']]
y = winner_data['winner']

# Convert categorical variables
X = pd.get_dummies(X, columns=['toss_winner'])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
winner_model = RandomForestClassifier(n_estimators=100, random_state=42)
winner_model.fit(X_train, y_train)

# Make predictions
y_pred = winner_model.predict(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy}')

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Model Accuracy: 0.3811881188118812

Classification Report:
                             precision    recall  f1-score   support

        Chennai Super Kings       0.44      0.54      0.48        28
            Deccan Chargers       0.29      0.50      0.36         4
             Delhi Capitals       0.20      0.25      0.22         8
           Delhi Daredevils       0.50      0.67      0.57        12
              Gujarat Lions       0.00      0.00      0.00         2
             Gujarat Titans       0.60      0.75      0.67         4
            Kings XI Punjab       0.23      0.20      0.21        15
      Kolkata Knight Riders       0.42      0.61      0.50        18
       Lucknow Super Giants       0.00      0.00      0.00         4
             Mumbai Indians       0.33      0.43      0.38        21
                         NA       0.00      0.00      0.00         1
              Pune Warriors       0.00      0.00      0.00         5
               Punjab Kings       0.50     

c:\DE\.venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\DE\.venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\DE\.venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [13]:
# Stop Spark Session
spark.stop()